# Extracción y descripción de los datos

Empiezo importando las librerías que voy a utilizar.

In [14]:
%matplotlib inline

import numpy as np
import pandas as pd

import librosa.display
import librosa

from glob import glob

import ffmpeg

import os

from pydub import AudioSegment
from pydub.utils import make_chunks

import re

Continuo creando 3 funciones con las que voy a hacer los siguiente:
 - La primera divide un archivo audio de larga duración en pequeños audios de 10 segundos de duración, y los guarda en la carpeta que le he especificado.
 - La segunda accede a cada uno de esos audios de 10 segundos y saca las features características de dicho archivo, y en base a su nombre le asigna una categoría y otra.
 - La tercera simplemente me enseña las principales características y métricas de la base de datos.

Finalmente, guardo el el dataset en mi carpeta como un CSV.

Dado que la base de datos ya la tengo creada, este JN lo voy a aplicar sobre audios que luego voy a utilizar para predecir con mi modelo y ver si es lo suficientemente bueno.

In [19]:
def decompose_files(data_dir, audio_files):
    for j in range(len(audio_files)):
        myaudio = AudioSegment.from_file(data_dir + '{}'.format(os.listdir(data_dir)[j])) 
        chunk_length_ms = 10000
        chunks = make_chunks(myaudio, chunk_length_ms)

        for i, chunk in enumerate(chunks):
            chunk_name = "{}{}.wav".format(os.listdir(data_dir)[j], i)
            print("exporting", chunk_name)
            chunk.export('./sounds2/{}'.format(chunk_name), format="mp3")
            
decompose_files('./sounds/', glob('./sounds' + '/*'))

exporting vacuum.mp30.wav
exporting vacuum.mp31.wav
exporting vacuum.mp32.wav
exporting vacuum.mp33.wav
exporting vacuum.mp34.wav
exporting vacuum.mp35.wav
exporting vacuum.mp36.wav
exporting vacuum_2.mp30.wav
exporting vacuum_2.mp31.wav
exporting vacuum_2.mp32.wav
exporting vacuum_2.mp33.wav
exporting vacuum_2.mp34.wav
exporting vacuum_2.mp35.wav
exporting vacuum_2.mp36.wav
exporting faucet.mp30.wav
exporting faucet.mp31.wav
exporting faucet.mp32.wav
exporting faucet.mp33.wav
exporting faucet.mp34.wav
exporting faucet.mp35.wav
exporting faucet.mp36.wav
exporting faucet_2.mp30.wav
exporting faucet_2.mp31.wav
exporting faucet_2.mp32.wav
exporting faucet_2.mp33.wav
exporting faucet_2.mp34.wav
exporting faucet_2.mp35.wav
exporting faucet_2.mp36.wav


In [20]:
def get_features(data_dir, audio_files):
    momentos = {'ducha': 'Ducha', 'cena': 'Cena', 'washing': 'Lavadora', 'vacuum': 'Aspiradora', 
                'shaver': 'Afeitadora', 'hairdryer': 'Secador_pelo', 'airconditioner': 'Aire_acondicionado', 
                'cellphone': 'Telefono', 'comp': 'Tecleo', 'silence': 'Silencio', 'dryer': 'Secadora', 
                'blender': 'Licuadora', 'doorbell': 'Timbre', 'alarm': 'Alarma', 'faucet': 'Grifo', 
                'microwave': 'Microondas'}
    features = []
    for i in range(len(audio_files)):
        y, sr = librosa.load(audio_files[i])
        name = os.listdir(data_dir)[i]
        pattern = "[._][\w]+"
        name = re.sub(pattern, '', name)
        if name in momentos.keys():
            momento = momentos.get(name)
        else:
            momento = 'Otro'
        mfcc = np.ndarray.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20))
        scem = np.ndarray.mean(librosa.feature.spectral_centroid(y=y, sr=sr))
        scom = np.ndarray.mean(librosa.feature.spectral_contrast(S=np.abs(librosa.stft(y)), sr=sr))
        srom = np.ndarray.mean(librosa.feature.spectral_rolloff(y=y, sr=sr))
        sbwm = np.ndarray.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))
        tempo = librosa.beat.tempo(onset_envelope=librosa.onset.onset_strength(y=y, sr=sr, hop_length=512), sr=sr, hop_length=512)[0]
        rmse = np.ndarray.mean(librosa.feature.rms(y=y))
        features.append([mfcc, scem, scom, srom, sbwm, tempo, rmse, momento])
    return pd.DataFrame(features, columns=['mfcc', 'scem','scom', 'srom','sbwm', 'tempo', 'rmse', 'momento'])

data = get_features('./sounds2/', glob('./sounds2' + '/*.wav'))

In [21]:
def show_df_info(dataframe):
    display("The type of the dataframe is {}.".format(type(dataframe)))
    print("------------------------------------------------------------------------")
    display("The amount of entries are {}.".format(dataframe.size))
    print("------------------------------------------------------------------------")
    display("Its shape is {}.".format(dataframe.shape))
    print("------------------------------------------------------------------------")
    display("Its features are: {}.".format(list(dataframe)))
    print("------------------------------------------------------------------------")
    print("The data types of columns are: {}".format(dataframe.dtypes))
    print("------------------------------------------------------------------------")
    print("The counter of each class is: {}".format(dataframe.momento.value_counts()))
    print("------------------------------------------------------------------------")
    print('The amount of null values is: {}.'.format(dataframe.isna().sum()))
    display(dataframe.head())
    display(dataframe.describe())
    
show_df_info(data)

"The type of the dataframe is <class 'pandas.core.frame.DataFrame'>."

------------------------------------------------------------------------


'The amount of entries are 224.'

------------------------------------------------------------------------


'Its shape is (28, 8).'

------------------------------------------------------------------------


"Its features are: ['mfcc', 'scem', 'scom', 'srom', 'sbwm', 'tempo', 'rmse', 'momento']."

------------------------------------------------------------------------
The data types of columns are: mfcc       float64
scem       float64
scom       float64
srom       float64
sbwm       float64
tempo      float64
rmse       float64
momento     object
dtype: object
------------------------------------------------------------------------
The counter of each class is: Grifo         14
Aspiradora    14
Name: momento, dtype: int64
------------------------------------------------------------------------
The amount of null values is: mfcc       0
scem       0
scom       0
srom       0
sbwm       0
tempo      0
rmse       0
momento    0
dtype: int64.


,mfcc,scem,scom,srom,sbwm,tempo,rmse,momento
0,-1.209925,3899.153153,18.999687,7674.313687,3091.657072,123.046875,0.153011,Aspiradora
1,-0.073909,4462.048199,19.394828,8128.784180,3059.540310,120.000000,0.225774,Aspiradora
2,0.091820,4143.076863,16.099284,8194.982542,3002.130473,123.046875,0.106121,Grifo
3,-1.693803,4342.919398,19.224401,7870.685508,3063.301995,123.046875,0.144743,Aspiradora
4,-9.215146,4809.931879,19.011070,8420.756428,3036.757352,123.046875,0.216905,Aspiradora


,mfcc,scem,scom,srom,sbwm,tempo,rmse
count,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000,28.000000
mean,-4.303088,4307.866445,17.994960,8092.243933,3014.115586,123.290762,0.121649
std,3.919068,314.678288,1.655027,404.215899,79.837520,2.842506,0.066696
min,-11.506991,3607.017904,15.780858,6607.970483,2733.730116,117.453835,0.024315
25%,-8.202848,4125.403339,16.238359,8027.922232,3002.659434,123.046875,0.054407
50%,-3.896869,4202.316503,18.727741,8236.625059,3034.369521,123.046875,0.109756
75%,-0.690039,4491.074084,19.035133,8311.685259,3040.364474,123.046875,0.167872
max,1.234239,4822.410675,21.697894,8447.185812,3184.399716,129.199219,0.225774


In [22]:
data.to_csv('data_prueba.csv', index=False)